In [1]:
import requests

def fetch_news_data(query):
    api_key = '23f5208b7e2940aeb8b72c67195565b2'
    date_from = '2024-01-15'
    date_to = '2024-02-14'
    sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

    url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'articles' in data:
        return [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    else:
        print("Error or no articles found. Response:")
        print(data)
        return []

# Broadening the search query
msft_data = fetch_news_data('Microsoft OR "MSFT" OR "Satya Nadella" OR "Windows" OR "Azure" OR "Office 365" OR "Xbox" OR "Surface" OR "Microsoft Teams" OR "LinkedIn"')

ibm_data = fetch_news_data('IBM OR "International Business Machines" OR "Arvind Krishna" OR "Watson" OR "Cloud computing" OR "Quantum computing" OR "IBM Research" OR "Artificial Intelligence" OR "Blockchain"')

amzn_data = fetch_news_data('Amazon OR "AMZN" OR "Jeff Bezos" OR "Andy Jassy" OR "E-commerce" OR "Amazon Web Services" OR "AWS" OR "Amazon Prime" OR "Amazon marketplace" OR "Whole Foods" OR "Amazon logistics"')

goog_data = fetch_news_data('Google OR "Alphabet Inc." OR "Sundar Pichai" OR "Google Search" OR "Android" OR "Stocks" OR "Google Cloud" OR "YouTube"  OR "Technology" OR "Google Ads" OR "Google services" OR "Google AI" OR "Pixel" OR "AI" OR "Artificial Intelligence" OR "Regulation" OR "Google Sustainability" OR "Google Health" OR "DeepMind" OR "Google Quantum Computing" OR "Google X" OR "Moonshot Factory"')

orcl_data = fetch_news_data('Oracle OR "ORCL" OR "Larry Ellison" OR "Safra Catz" OR "Enterprise software" OR "Cloud services" OR "Database technology" OR "Oracle financials" OR "Technology" OR "Tech" OR "Java" OR "DB" OR "Database" OR "Databases" OR "Relational Databases" OR "Software" OR "Computer" OR "Systems" OR "AWS" OR "Google Cloud" OR "Google Cloud Bucket" OR "Azure" OR "Microsoft"')

nvda_data = fetch_news_data('NVDA OR "NVIDIA Corporation" OR "Graphics Processing Units" OR "Jensen Huang" '
         'OR "Artificial Intelligence" OR "Gaming Industry" OR "Computer Graphics" '
         'OR "NVIDIA products" OR "NVIDIA stock" OR "Semiconductor industry" '
         'OR "Deep learning" OR "GPU technology" OR "NVIDIA partnerships" '
         'OR "NVIDIA competition" OR "NVIDIA financials" OR "Tech industry" '
         'OR "Data centers" OR "Autonomous vehicles" OR "Gaming GPUs" '
         'OR "NVIDIA acquisitions" OR "Ray tracing technology"')

fdx_data = fetch_news_data('FedEx OR "FDX" OR "Frederick W. Smith" OR "Express delivery" OR "Courier services" '
         'OR "Logistics services" OR "FedEx Ground" OR "FedEx Freight" OR "Shipping industry" '
         'OR "Parcel delivery" OR "E-commerce delivery" OR "Supply chain" OR "FedEx logistics" '
         'OR "FedEx innovations" OR "Delivery technology" OR "FedEx and ecommerce" '
         'OR "FedEx sustainability" OR "Delivery drones" OR "FedEx automation"')

abnb_data = fetch_news_data('Airbnb OR "ABNB" OR "Short-term rental" OR "Brian Chesky" OR "Vacation rental" '
         'OR "Airbnb experiences" OR "Airbnb IPO" OR "Home sharing" OR "Travel industry" '
         'OR "Airbnb regulations" OR "Hospitality industry" OR "Airbnb market" OR "Platform economy" '
         'OR "Sharing economy" OR "Airbnb competitors" OR "Airbnb legal" OR "Airbnb challenges" '
         'OR "Tourism disruption" OR "Airbnb strategy" OR "Airbnb expansion" OR "Peer-to-peer lodging" '
         'OR "technology" OR "istings" OR "tourism" OR "Travel" OR "Remote work"')

tsla_data = fetch_news_data('Elon Musk OR Tesla OR "Electric Vehicles" OR "SpaceX" OR "Neuralink" OR "Twitter acquisition" OR "Tesla stock" OR "Autonomous driving" OR "Battery technology" OR "Solar energy" OR "Hyperloop" OR "Mars colonization"')

appl_data = fetch_news_data('Apple OR "Apple Inc." OR "iPhone" OR "iPad" OR "Mac" OR "Apple Watch" OR "iOS" OR "App Store" OR "MacOS" OR "Apple Music" OR "Tim Cook"')


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def analyze_sentiment(news_data):
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
    model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

    # Extract just the headlines
    headlines = [headline for headline, _ in news_data]

    # Tokenize and encode headlines
    max_length = 512
    inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

    # Predict sentiment
    sentiment_data = []
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

        # Pair each sentiment score with its corresponding date
        for (headline, date), prediction in zip(news_data, predictions):
            sentiment_data.append({
                'date': date,
                'headline': headline,
                'sentiment_positive': prediction[0].item(),
                'sentiment_neutral': prediction[1].item(),
                'sentiment_negative': prediction[2].item()
            })

    return sentiment_data

In [3]:
# Analyzing sentiment for each company
msft_sentiment = analyze_sentiment(msft_data)
ibm_sentiment = analyze_sentiment(ibm_data)
amzn_sentiment = analyze_sentiment(amzn_data)
goog_sentiment = analyze_sentiment(goog_data)
orcl_sentiment = analyze_sentiment(orcl_data)
nvda_sentiment = analyze_sentiment(nvda_data)
fdx_sentiment = analyze_sentiment(fdx_data)
abnb_sentiment = analyze_sentiment(abnb_data)
tsla_sentiment = analyze_sentiment(tsla_data)
appl_sentiment = analyze_sentiment(appl_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [4]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

def process_stock_sentiment(sentiment_data, ticker_symbol, start_date, end_date):
    # Convert sentiment_data to a DataFrame
    sentiment_df = pd.DataFrame(sentiment_data)

    # Convert date to datetime and set as index
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
    sentiment_df.set_index('date', inplace=True)

    # Aggregate sentiment scores by date
    average_sentiment = sentiment_df.groupby('date').mean()

    # Download stock data
    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

    # Selecting the required columns
    stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

    # Convert stock_data index to datetime
    stock_data.index = pd.to_datetime(stock_data.index)

    # Combine stock data with sentiment data
    combined_data = stock_data.join(average_sentiment)

    # Interpolate missing values
    combined_data = combined_data.interpolate(method='time')

    # Scale price-related columns
    price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
    scaler_prices = StandardScaler()
    combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

    # Scale Volume column independently
    scaler_volume = StandardScaler()
    combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

    return combined_data, scaler_prices

# Process Microsoft's sentiment and stock data
msft_combined_data = process_stock_sentiment(msft_sentiment, 'MSFT', '2024-01-15', '2024-02-14')

ibm_combined_data = process_stock_sentiment(ibm_sentiment, 'IBM', '2024-01-15', '2024-02-14')

amzn_combined_data = process_stock_sentiment(amzn_sentiment, 'AMZN', '2024-01-15', '2024-02-14')

goog_combined_data = process_stock_sentiment(goog_sentiment, 'GOOG', '2024-01-15', '2024-02-14')

orcl_combined_data = process_stock_sentiment(orcl_sentiment, 'ORCL', '2024-01-15', '2024-02-14')

nvda_combined_data = process_stock_sentiment(nvda_sentiment, 'NVDA', '2024-01-15', '2024-02-14')

fdx_combined_data = process_stock_sentiment(fdx_sentiment, 'FDX', '2024-01-15', '2024-02-14')

abnb_combined_data = process_stock_sentiment(abnb_sentiment, 'ABNB', '2024-01-15', '2024-02-14')

tsla_combined_data = process_stock_sentiment(tsla_sentiment, 'TSLA', '2024-01-15', '2024-02-14')

appl_combined_data = process_stock_sentiment(appl_sentiment, 'AAPL', '2024-01-15', '2024-02-14')

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()
<ipython-input-4-63cfab7b44c3>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()
[*********************100%%**********************]  1 of 1 completed
<ipython-input-4-63cfab7b44c3>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()
[******************

# Combined Data for Every Company (Just Checking DataFrames):

Checking to see if there's any NULL

In [5]:
msft_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.308886,-1.596025,-1.666334,-1.740528,-1.740528,0.260285,0.999958,3.224848e-05,0.000010
2024-01-17,-2.027688,-2.064255,-2.014565,-1.839413,-1.839413,-0.519244,0.719211,2.801329e-01,0.000656
2024-01-18,-1.554393,-1.600804,-1.356518,-1.295536,-1.295536,-0.337549,0.999991,3.105806e-07,0.000008
2024-01-19,-1.043131,-1.041794,-0.937648,-0.702213,-0.702213,0.585029,0.523730,4.671861e-01,0.009084
2024-01-22,-0.504031,-0.808877,-0.926495,-0.969208,-0.969208,0.231195,0.957338,2.153970e-03,0.040508
2024-01-23,-1.044397,-0.956989,-0.884360,-0.673785,-0.673785,-0.787267,0.720906,2.785722e-01,0.000522
2024-01-24,-0.311673,-0.210452,-0.076360,-0.221377,-0.221377,-0.106132,0.845814,1.499272e-03,0.152686
2024-01-25,0.040135,-0.045616,0.181404,0.064159,0.064159,-0.709553,0.620905,2.201809e-01,0.158915
2024-01-26,0.046461,-0.145950,0.169011,-0.052034,-0.052034,-1.214453,0.755356,2.136181e-01,0.031026


In [6]:
ibm_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.868356,-1.740358,-1.830657,-1.769062,-1.761494,-0.307363,0.369822,0.000083,0.630094
2024-01-17,-1.741923,-1.673615,-1.809198,-1.883275,-1.872598,-0.413979,0.998427,0.000162,0.001411
2024-01-18,-1.780235,-1.770800,-1.870893,-1.784638,-1.776646,-0.507879,0.735575,0.234835,0.029590
2024-01-19,-1.256620,-1.233355,-1.315642,-1.182425,-1.190826,0.069961,0.749402,0.249683,0.000915
2024-01-22,-0.971822,-0.897305,-0.883779,-1.007211,-1.020382,-0.297013,0.854732,0.005886,0.139382
2024-01-23,-0.961607,-0.947653,-0.873050,-0.863147,-0.880240,-0.469967,0.744501,0.002146,0.253353
2024-01-24,-0.724064,-0.849298,-0.816720,-0.864446,-0.881505,0.468534,0.999878,0.000023,0.000099
2024-01-25,0.578594,1.731380,0.783318,1.277045,1.201688,4.230100,0.566644,0.054521,0.378835
2024-01-26,1.389560,1.203302,0.961696,0.886385,0.821663,0.614990,0.507017,0.219305,0.273678


In [7]:
amzn_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.098181,-1.094539,-1.101233,-1.176551,-1.176551,-0.576020,0.999997,3.759072e-07,0.000003
2024-01-17,-1.380485,-1.473867,-1.416271,-1.371575,-1.371575,-0.952045,0.805210,1.932141e-01,0.001575
2024-01-18,-1.203353,-1.256154,-1.147643,-1.130821,-1.130821,-0.782672,0.558725,3.339560e-01,0.107319
2024-01-19,-1.056665,-0.991695,-1.018252,-0.883342,-0.883342,-0.011847,0.501150,4.988454e-01,0.000004
2024-01-22,-0.633209,-0.819394,-0.855108,-0.958662,-0.958662,-0.441371,0.652264,1.436822e-02,0.333367
2024-01-23,-0.915512,-0.931588,-0.850889,-0.791882,-0.791882,-0.774732,0.703865,1.040968e-01,0.192038
2024-01-24,-0.507278,-0.624389,-0.492250,-0.677558,-0.677558,-0.157224,0.795875,1.133019e-04,0.204012
2024-01-25,-0.624906,-0.624389,-0.763689,-0.559198,-0.559198,-0.444231,0.999920,8.074728e-06,0.000072
2024-01-26,-0.421480,-0.329207,-0.291130,-0.374934,-0.374934,-0.011046,0.910538,5.600947e-05,0.089406


In [8]:
goog_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.046810,-0.806770,-0.777919,-0.925480,-0.925480,-0.536688,0.404725,0.112852,0.482423
2024-01-17,-1.190315,-1.479196,-1.408058,-1.238356,-1.238356,-0.715514,0.747762,0.160961,0.091277
2024-01-18,-1.044047,-0.877330,-0.705152,-0.686221,-0.686221,-0.580510,0.335727,0.210353,0.453919
2024-01-19,-0.253375,-0.197988,-0.098776,0.097281,0.097281,0.547916,0.718644,0.009257,0.272099
2024-01-22,0.410354,0.348535,0.341776,0.028923,0.028923,-0.178831,0.999928,0.000059,0.000012
2024-01-23,0.137135,0.028924,0.245251,0.283952,0.283952,-1.228552,0.647391,0.020177,0.332432
2024-01-24,0.846394,0.778835,0.901127,0.723033,0.723033,-0.530416,0.528263,0.002908,0.468828
2024-01-25,1.246565,1.661566,1.242679,1.588040,1.588040,-0.224286,0.529836,0.183492,0.286673
2024-01-26,1.558416,1.481701,1.633731,1.627476,1.627476,-0.496471,0.784434,0.057472,0.158094


In [9]:
orcl_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.971241,-2.250502,-2.084669,-2.098505,-2.098505,-0.060310,0.999869,1.584024e-06,0.000129
2024-01-17,-1.956482,-1.997773,-2.005542,-2.141949,-2.141949,0.058880,0.999040,1.799734e-04,0.000780
2024-01-18,-1.800026,-1.582574,-1.719470,-1.437523,-1.437523,1.435591,0.152284,8.475493e-01,0.000167
2024-01-19,-1.112206,-1.179408,-0.989077,-1.136512,-1.136512,1.838912,0.717854,1.870258e-01,0.095120
2024-01-22,-0.893758,-1.037999,-0.912995,-1.003074,-1.003074,-0.093202,0.466403,5.334885e-01,0.000108
2024-01-23,-0.825861,-0.439269,-0.824738,-0.466219,-0.466219,0.810470,0.597226,7.177080e-02,0.331003
2024-01-24,0.098119,0.385113,0.322592,0.303375,0.303375,2.147859,0.827621,1.681252e-03,0.170697
2024-01-25,0.532065,0.613774,0.633008,0.517496,0.517496,0.646432,0.518940,3.101387e-01,0.170921
2024-01-26,0.458265,0.403166,0.608661,0.405781,0.405781,-1.125948,0.810253,1.727012e-01,0.017046


In [10]:
nvda_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.628293,-1.417470,-1.512552,-1.437650,-1.437650,-0.323581,0.369822,8.340228e-05,0.630094
2024-01-17,-1.369936,-1.483936,-1.544327,-1.500394,-1.500394,-0.075395,0.605685,1.927488e-01,0.201566
2024-01-18,-1.192449,-1.277780,-1.272840,-1.299384,-1.299384,0.097198,0.867079,2.341395e-02,0.109507
2024-01-19,-1.050731,-0.930838,-1.050805,-0.844727,-0.844727,0.601823,0.589669,1.669669e-01,0.243364
2024-01-22,-0.650268,-0.779097,-0.684387,-0.813641,-0.813641,-0.289825,0.464425,4.265030e-01,0.109072
2024-01-23,-0.743385,-0.855973,-0.780709,-0.771875,-0.771875,-1.873132,0.666341,1.064737e-03,0.332594
2024-01-24,-0.600696,-0.319308,-0.512002,-0.487905,-0.487905,0.783537,0.993119,5.472353e-04,0.006333
2024-01-25,-0.202953,-0.343046,-0.330878,-0.439274,-0.439274,0.008451,0.489592,1.101037e-01,0.400304
2024-01-26,-0.473170,-0.513960,-0.385891,-0.551031,-0.551031,-0.916403,0.703842,5.171567e-02,0.244443


In [11]:
fdx_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,0.521559,0.144192,0.207173,0.071969,0.071969,0.938509,6.317959e-01,0.209660,1.585439e-01
2024-01-17,-0.684943,-0.425300,-0.285093,-0.328776,-0.328776,-0.734148,6.611221e-01,0.000488,3.383902e-01
2024-01-18,-0.013898,-0.395204,-0.171143,-0.150155,-0.150155,-0.564989,4.785720e-01,0.001418,5.200100e-01
2024-01-19,-0.043775,0.005290,-0.166584,0.413176,0.413176,0.205979,2.319877e-01,0.000124,7.678882e-01
2024-01-22,0.556030,1.181311,1.059520,1.365804,1.365804,0.000796,5.851925e-01,0.006811,4.079966e-01
2024-01-23,1.424711,1.206777,1.492534,1.402444,1.402444,-1.236428,4.823953e-01,0.000332,5.172725e-01
2024-01-24,1.652224,1.595699,1.376302,1.145965,1.145965,0.309963,4.599268e-01,0.272380,2.676934e-01
2024-01-25,1.578687,1.903594,1.984800,1.924557,1.924557,-1.086394,6.278665e-01,0.000917,3.712163e-01
2024-01-26,1.939487,1.808678,1.829826,1.562741,1.562741,-1.009706,4.016374e-01,0.200008,3.983548e-01


In [12]:
abnb_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.718478,-1.887552,-1.721813,-1.867322,-1.867322,0.197382,0.995512,0.000008,0.004480
2024-01-17,-2.365285,-2.051583,-2.244501,-1.874875,-1.874875,-0.862138,0.757275,0.242549,0.000176
2024-01-18,-1.485965,-1.250356,-1.405554,-1.485975,-1.485975,0.093874,0.967929,0.000051,0.032020
2024-01-19,-1.147764,-1.115166,-1.182852,-0.940383,-0.940383,-0.414546,0.998703,0.000994,0.000303
2024-01-22,-0.607703,-0.436513,-0.251370,-0.547706,-0.547706,-0.244345,0.797070,0.003245,0.199685
2024-01-23,-0.270560,-0.458145,-0.328653,-0.492957,-0.492957,-0.501530,0.503558,0.000022,0.496420
2024-01-24,-0.046503,-0.427500,-0.519831,-0.702510,-0.702510,-0.084118,0.797981,0.001704,0.200315
2024-01-25,-0.175442,-0.573507,-0.359568,-0.528827,-0.528827,-0.589114,0.456218,0.441433,0.102349
2024-01-26,-0.405839,0.911780,-0.131373,0.888959,0.888959,2.692096,0.605153,0.250714,0.144133


In [13]:
tsla_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,1.506231,1.908116,1.627338,1.977109,1.977109,0.164797,0.486740,0.147426,0.365834
2024-01-17,1.487403,1.291841,1.613722,1.623083,1.623083,-0.353643,0.609293,0.389932,0.000776
2024-01-18,1.645871,1.432118,1.351819,1.325084,1.325084,-0.122674,0.999813,0.000019,0.000168
2024-01-19,1.105354,1.096399,1.257308,1.350256,1.350256,-0.399088,0.923613,0.063665,0.012722
2024-01-22,1.283434,1.459701,1.153989,1.074993,1.074993,0.275263,0.767855,0.000001,0.232143
2024-01-23,1.208123,1.290265,1.272526,1.102600,1.102600,-0.207283,0.995361,0.000027,0.004611
2024-01-24,1.253624,1.060147,1.194035,0.996230,0.996230,0.505654,0.670388,0.329499,0.000113
2024-01-25,-0.486387,-0.494723,-0.945251,-1.049975,-1.049975,3.682779,0.397428,0.000420,0.602152
2024-01-26,-0.815875,-0.984905,-0.781860,-0.999632,-0.999632,-0.175927,0.918070,0.000022,0.081907


In [14]:
appl_combined_data[0]

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2024-01-16,-1.476815,-1.593173,-1.498245,-1.496294,-1.513302,0.613420,0.660642,6.706586e-02,0.272292
2024-01-17,-1.677643,-1.944348,-1.647088,-1.760326,-1.778652,-0.655456,0.998598,4.214902e-04,0.000980
2024-01-18,-0.590018,-0.304653,-0.340555,-0.106669,-0.116723,1.474076,0.498843,4.743342e-01,0.026823
2024-01-19,0.141084,0.437301,0.365871,0.707650,0.701667,0.831172,0.840369,1.563685e-01,0.003262
2024-01-22,0.811261,1.329760,1.178612,1.355215,1.352470,0.233908,0.907337,9.248612e-02,0.000176
2024-01-23,1.425025,1.440657,1.549546,1.713737,1.712788,-0.999766,0.996675,6.279634e-04,0.002697
2024-01-24,1.515283,1.607004,1.670039,1.524750,1.522852,-0.217322,0.855397,1.135638e-03,0.143468
2024-01-25,1.470154,1.577959,1.379437,1.433034,1.430679,-0.134689,0.500006,2.060314e-01,0.293963
2024-01-26,1.255789,1.179255,1.103010,0.946665,0.941877,-0.844439,0.671558,3.284095e-01,0.000033


# Processing data into model

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Input, MultiHeadAttention, GlobalAveragePooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

def transformer_model(combined_data, scaler_prices, company_name):
  selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
  data = combined_data[selected_features]

  train_size = int(len(data) * 0.7)
  val_size = int(len(data) * 0.15)
  train_data = data.iloc[:train_size]
  val_data = data.iloc[train_size:train_size + val_size]
  test_data = data.iloc[train_size + val_size:]

  # Prepare features and target for model training
  X_train = train_data.drop('Close', axis=1)
  y_train = train_data['Close']
  X_val = val_data.drop('Close', axis=1)
  y_val = val_data['Close']
  X_test = test_data.drop('Close', axis=1)
  y_test = test_data['Close']

  # Define and compile the neural network model
  def create_model(input_shape):
      inp = Input(shape=input_shape)

      # LSTM Layer
      x = LSTM(128, return_sequences=True)(inp)
      x = Dropout(0.2)(x)

      # Transformer Layer using MultiHeadAttention
      attention = MultiHeadAttention(num_heads=2, key_dim=128)(x, x)
      x = GlobalAveragePooling1D()(attention)

      # Dense Layers for final predictions
      x = Dense(64, activation='LeakyReLU')(x)
      x = Dropout(0.2)(x)
      x = Dense(32, activation='LeakyReLU')(x)
      x = Dense(1)(x)

      model = Model(inputs=inp, outputs=x)
      model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
      return model

  # Reshape the data for LSTM layer
  X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
  X_val_reshaped = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))
  X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

  # Train the model
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

  # Train the model with early stopping
  model = create_model((1, X_train.shape[1]))
  history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=8, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])

  train_loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(train_loss) + 1)

  # Convert the epochs range object to a list
  epochs_list = list(epochs)

  # Plotting the training and validation loss
  loss_fig = go.Figure()
  loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name=f'{company_name} Training Loss'))
  loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name=f'{company_name} Validation Loss'))
  loss_fig.update_layout(title=f'{company_name} Training and Validation Loss per Epoch',
                      xaxis_title='Epochs',
                      yaxis_title='Loss',
                      legend_title='Type')
  loss_fig.show()

  # Make predictions
  # Reshape X_test for prediction
  X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

  # Make predictions using the reshaped test data
  predictions = model.predict(X_test_reshaped)

  # Evaluate the model
  mse = mean_squared_error(y_test, predictions)
  mae = mean_absolute_error(y_test, predictions)
  r2 = r2_score(y_test, predictions)
  print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

  # Reshape y_test and predictions for inverse transform
  y_test_reshaped = y_test.values.reshape(-1, 1)
  predictions_reshaped = predictions.reshape(-1, 1)

  num_scaled_cols = 5

  # Create separate dummy arrays for inverse scaling
  dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
  dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

  # Fill in the 'Close' column values in the dummy arrays
  # Assuming 'Close' is the last of the scaled columns
  dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
  dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

  # Inverse transform the 'Close' prices using the dummy arrays
  y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
  predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

  # Extracting testing dates
  test_dates = test_data.index

  # Plotting with Plotly
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name=f'Actual {company_name}'))
  fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name=f'Predicted {company_name}'))
  fig.update_layout(title=f'Actual vs Predicted {company_name} Stock Prices',
                    xaxis_title='Date',
                    yaxis_title='Stock Price',
                    legend_title='Legend')
  fig.show()

  return model

# Plots from models:

In [16]:
msft_model = transformer_model(msft_combined_data[0], msft_combined_data[1], 'MSFT')

msft_model

Epoch 1/50
2/2 [==============================] - 3s 400ms/step - loss: 0.7249 - val_loss: 0.3833
Epoch 2/50
2/2 [==============================] - 0s 21ms/step - loss: 4.7975 - val_loss: 0.4373
Epoch 3/50
2/2 [==============================] - 0s 21ms/step - loss: 0.8944 - val_loss: 0.4411
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6236 - val_loss: 0.3665
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.3007 - val_loss: 0.2031
Epoch 6/50
2/2 [==============================] - 0s 25ms/step - loss: 0.1447 - val_loss: 0.1755
Epoch 7/50
2/2 [==============================] - 0s 24ms/step - loss: 0.2280 - val_loss: 0.1558
Epoch 8/50
2/2 [==============================] - 0s 26ms/step - loss: 0.3938 - val_loss: 0.3008
Epoch 9/50
2/2 [==============================] - 0s 24ms/step - loss: 0.2523 - val_loss: 0.1685
Epoch 10/50
2/2 [==============================] - 0s 25ms/step - loss: 0.1488 - val_loss: 0.0448
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 314ms/step
MSE: 0.28843486778788124, MAE: 0.5081631071555693, R^2: 0.2702130682922058


In [17]:
ibm_model = transformer_model(ibm_combined_data[0], ibm_combined_data[1], 'IBM')

ibm_model

Epoch 1/50
2/2 [==============================] - 3s 368ms/step - loss: 0.9165 - val_loss: 0.0265
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 6.6276 - val_loss: 0.0022
Epoch 3/50
2/2 [==============================] - 0s 22ms/step - loss: 1.0733 - val_loss: 0.0306
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.6382 - val_loss: 0.1000
Epoch 5/50
2/2 [==============================] - 0s 24ms/step - loss: 0.3994 - val_loss: 0.0320
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.2151 - val_loss: 0.3280
Epoch 7/50
2/2 [==============================] - 0s 33ms/step - loss: 0.2685 - val_loss: 0.2278
Epoch 7: early stopping


1/1 [==============================] - 0s 292ms/step
MSE: 0.06702816360589524, MAE: 0.22500137675599874, R^2: -2.08319253678527


In [18]:
amzn_model = transformer_model(amzn_combined_data[0], amzn_combined_data[1], 'AMZN')

amzn_model

Epoch 1/50
2/2 [==============================] - 2s 393ms/step - loss: 0.5580 - val_loss: 0.0102
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 6.8494 - val_loss: 1.5057
Epoch 3/50
2/2 [==============================] - 0s 28ms/step - loss: 0.8012 - val_loss: 1.2906
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 0.4872 - val_loss: 1.3908
Epoch 5/50
2/2 [==============================] - 0s 22ms/step - loss: 0.2638 - val_loss: 1.2358
Epoch 6/50
2/2 [==============================] - 0s 30ms/step - loss: 0.2552 - val_loss: 0.9123
Epoch 6: early stopping


1/1 [==============================] - 0s 312ms/step
MSE: 0.23385877206878683, MAE: 0.396052196010669, R^2: -1.558609298296727


In [19]:
goog_model = transformer_model(goog_combined_data[0], goog_combined_data[1], 'GOOG')

goog_model

Epoch 1/50
2/2 [==============================] - 2s 375ms/step - loss: 1.2208 - val_loss: 0.1568
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 2.3612 - val_loss: 0.0907
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 0.9292 - val_loss: 0.0207
Epoch 4/50
2/2 [==============================] - 0s 22ms/step - loss: 0.3659 - val_loss: 0.1337
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.1352 - val_loss: 0.0882
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.1053 - val_loss: 0.0233
Epoch 7/50
2/2 [==============================] - 0s 25ms/step - loss: 0.4565 - val_loss: 0.0143
Epoch 8/50
2/2 [==============================] - 0s 23ms/step - loss: 0.1891 - val_loss: 0.0705
Epoch 9/50
2/2 [==============================] - 0s 23ms/step - loss: 0.3047 - val_loss: 0.0473
Epoch 10/50
2/2 [==============================] - 0s 22ms/step - loss: 0.2500 - val_loss: 0.0382
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 304ms/step
MSE: 0.017455857194069783, MAE: 0.11046096398034833, R^2: 0.8832788064359749


In [20]:
orcl_model = transformer_model(orcl_combined_data[0], orcl_combined_data[1], 'ORCL')

orcl_model

Epoch 1/50
2/2 [==============================] - 3s 440ms/step - loss: 1.0354 - val_loss: 0.8283
Epoch 2/50
2/2 [==============================] - 0s 29ms/step - loss: 1.2227 - val_loss: 0.1489
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 0.0507 - val_loss: 0.0851
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.1240 - val_loss: 0.2666
Epoch 5/50
2/2 [==============================] - 0s 27ms/step - loss: 0.6741 - val_loss: 3.3653
Epoch 6/50
2/2 [==============================] - 0s 25ms/step - loss: 0.6730 - val_loss: 0.0223
Epoch 7/50
2/2 [==============================] - 0s 28ms/step - loss: 0.6260 - val_loss: 0.1157
Epoch 8/50
2/2 [==============================] - 0s 26ms/step - loss: 0.2646 - val_loss: 0.2733
Epoch 9/50
2/2 [==============================] - 0s 29ms/step - loss: 0.1922 - val_loss: 0.1295
Epoch 10/50
2/2 [==============================] - 0s 30ms/step - loss: 0.1702 - val_loss: 0.0132
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 319ms/step
MSE: 0.024847169286088645, MAE: 0.1402856262774525, R^2: 0.8263531346930519


In [21]:
nvda_model = transformer_model(nvda_combined_data[0], nvda_combined_data[1], 'NVDA')

nvda_model

Epoch 1/50
2/2 [==============================] - 2s 361ms/step - loss: 0.6356 - val_loss: 0.4461
Epoch 2/50
2/2 [==============================] - 0s 25ms/step - loss: 0.4164 - val_loss: 1.4762
Epoch 3/50
2/2 [==============================] - 0s 25ms/step - loss: 0.5251 - val_loss: 1.6837
Epoch 4/50
2/2 [==============================] - 0s 23ms/step - loss: 0.3585 - val_loss: 2.2880
Epoch 5/50
2/2 [==============================] - 0s 25ms/step - loss: 0.2301 - val_loss: 3.0237
Epoch 6/50
2/2 [==============================] - 0s 29ms/step - loss: 0.1488 - val_loss: 2.3810
Epoch 6: early stopping


1/1 [==============================] - 0s 298ms/step
MSE: 0.2270299879808222, MAE: 0.43305758037025427, R^2: -4.196461834638158


In [22]:
fdx_model = transformer_model(fdx_combined_data[0], fdx_combined_data[1], 'FDX')

fdx_model

Epoch 1/50
2/2 [==============================] - 2s 381ms/step - loss: 0.5393 - val_loss: 0.9126
Epoch 2/50
2/2 [==============================] - 0s 23ms/step - loss: 16.8232 - val_loss: 0.7935
Epoch 3/50
2/2 [==============================] - 0s 27ms/step - loss: 1.0223 - val_loss: 0.6697
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 0.7323 - val_loss: 0.5996
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.4948 - val_loss: 0.5883
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.1684 - val_loss: 0.2708
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 0.3344 - val_loss: 0.0652
Epoch 8/50
2/2 [==============================] - 0s 24ms/step - loss: 0.3498 - val_loss: 0.1105
Epoch 9/50
2/2 [==============================] - 0s 21ms/step - loss: 0.4354 - val_loss: 0.3068
Epoch 10/50
2/2 [==============================] - 0s 23ms/step - loss: 0.1553 - val_loss: 0.3872
Epoch 11/50
2/2 [==========

1/1 [==============================] - 0s 312ms/step
MSE: 0.020844306365938824, MAE: 0.1287097103926065, R^2: 0.9557267780851552


In [23]:
abnb_model = transformer_model(abnb_combined_data[0], abnb_combined_data[1], 'ABNB')

abnb_model

Epoch 1/50
2/2 [==============================] - 3s 550ms/step - loss: 0.7079 - val_loss: 0.7118
Epoch 2/50
2/2 [==============================] - 0s 25ms/step - loss: 6.1338 - val_loss: 0.0556
Epoch 3/50
2/2 [==============================] - 0s 24ms/step - loss: 0.8607 - val_loss: 0.1023
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.7329 - val_loss: 0.1433
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.5381 - val_loss: 0.0951
Epoch 6/50
2/2 [==============================] - 0s 24ms/step - loss: 0.3732 - val_loss: 0.0838
Epoch 7/50
2/2 [==============================] - 0s 29ms/step - loss: 0.2684 - val_loss: 0.1234
Epoch 7: early stopping


1/1 [==============================] - 0s 296ms/step
MSE: 0.768867588227496, MAE: 0.794825422508066, R^2: -3.473275390696709


In [24]:
tsla_model = transformer_model(tsla_combined_data[0], tsla_combined_data[1], 'TSLA')

tsla_model

Epoch 1/50
2/2 [==============================] - 3s 380ms/step - loss: 0.7123 - val_loss: 1.2315
Epoch 2/50
2/2 [==============================] - 0s 26ms/step - loss: 11.6585 - val_loss: 0.3112
Epoch 3/50
2/2 [==============================] - 0s 24ms/step - loss: 1.3344 - val_loss: 0.3819
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.9159 - val_loss: 0.2357
Epoch 5/50
2/2 [==============================] - 0s 25ms/step - loss: 0.5931 - val_loss: 0.0331
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.2225 - val_loss: 0.0829
Epoch 7/50
2/2 [==============================] - 0s 24ms/step - loss: 0.1582 - val_loss: 0.0287
Epoch 8/50
2/2 [==============================] - 0s 24ms/step - loss: 0.3097 - val_loss: 0.0024
Epoch 9/50
2/2 [==============================] - 0s 25ms/step - loss: 0.1242 - val_loss: 0.0234
Epoch 10/50
2/2 [==============================] - 0s 28ms/step - loss: 0.0933 - val_loss: 0.1364
Epoch 11/50
2/2 [==========

1/1 [==============================] - 0s 293ms/step
MSE: 0.011831698266968223, MAE: 0.08954919014181287, R^2: 0.8460500435025375


In [25]:
appl_model = transformer_model(appl_combined_data[0], appl_combined_data[1], 'APPL')

appl_model

Epoch 1/50
2/2 [==============================] - 2s 375ms/step - loss: 1.0409 - val_loss: 0.7643
Epoch 2/50
2/2 [==============================] - 0s 27ms/step - loss: 2.1042 - val_loss: 0.1030
Epoch 3/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0136 - val_loss: 0.3065
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.7205 - val_loss: 0.2080
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.0763 - val_loss: 0.7329
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.7560 - val_loss: 0.1240
Epoch 7/50
2/2 [==============================] - 0s 29ms/step - loss: 0.4473 - val_loss: 0.1109
Epoch 7: early stopping


1/1 [==============================] - 0s 289ms/step
MSE: 0.330817631512623, MAE: 0.41864052170817645, R^2: -0.9997766291858203


# Federated Learning

In [28]:
import numpy as np

# Assuming you have these models already trained
existing_models = [msft_model, ibm_model, amzn_model, goog_model, orcl_model, nvda_model, fdx_model, abnb_model, tsla_model, appl_model]

def average_model_weights(models):
    # Filter out any None values from the models list
    valid_models = [model for model in models if model is not None]

    if not valid_models:
        raise ValueError("No valid models provided for averaging weights.")

    num_layers = len(valid_models[0].layers)
    average_weights = []

    for layer in range(num_layers):
        layer_weights_all_models = []

        # Collect weights for this layer from all models
        for model in valid_models:
            layer_weights = model.layers[layer].get_weights()
            if layer_weights:  # Ensure there are weights to process
                layer_weights_all_models.append(layer_weights)

        if not layer_weights_all_models:
            continue  # Skip layers with no weights

        # Calculate the average weights for this layer
        num_weights = len(layer_weights_all_models[0])  # Number of weight arrays in the layer (usually 2: weights and biases)
        avg_layer_weights = []
        for i in range(num_weights):
            # Calculate the mean for each weight array across all models
            weights_i = np.mean([model_weights[i] for model_weights in layer_weights_all_models], axis=0)
            avg_layer_weights.append(weights_i)

        average_weights.append(avg_layer_weights)

    # Flatten the list of averaged weights for setting in a model
    average_weights_flat = [weight for layer_weights in average_weights for weight in layer_weights]
    return average_weights_flat

avg_weights = average_model_weights(existing_models)

def transformer_avg_model(combined_data, scaler_prices, company_name, avg_weights):
  selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
  data = combined_data[selected_features]

  train_size = int(len(data) * 0.7)
  val_size = int(len(data) * 0.15)
  train_data = data.iloc[:train_size]
  val_data = data.iloc[train_size:train_size + val_size]
  test_data = data.iloc[train_size + val_size:]

  # Prepare features and target for model training
  X_train = train_data.drop('Close', axis=1)
  y_train = train_data['Close']
  X_val = val_data.drop('Close', axis=1)
  y_val = val_data['Close']
  X_test = test_data.drop('Close', axis=1)
  y_test = test_data['Close']

  # Define and compile the neural network model
  def create_model(input_shape):
      inp = Input(shape=input_shape)

      # LSTM Layer
      x = LSTM(128, return_sequences=True)(inp)
      x = Dropout(0.2)(x)

      # Transformer Layer using MultiHeadAttention
      attention = MultiHeadAttention(num_heads=2, key_dim=128)(x, x)
      x = GlobalAveragePooling1D()(attention)

      # Dense Layers for final predictions
      x = Dense(64, activation='LeakyReLU')(x)
      x = Dropout(0.2)(x)
      x = Dense(32, activation='LeakyReLU')(x)
      x = Dense(1)(x)

      model = Model(inputs=inp, outputs=x)
      model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
      return model

  # Reshape the data for LSTM layer
  X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
  X_val_reshaped = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))
  X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

  # Train the model
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

  # Train the model with early stopping
  model = create_model((1, X_train.shape[1]))
  model.set_weights(avg_weights)
  history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=8, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])

  train_loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(train_loss) + 1)

  # Convert the epochs range object to a list
  epochs_list = list(epochs)

  # Plotting the training and validation loss
  loss_fig = go.Figure()
  loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name=f'{company_name} Training Loss'))
  loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name=f'{company_name} Validation Loss'))
  loss_fig.update_layout(title=f'{company_name} Training and Validation Loss per Epoch',
                      xaxis_title='Epochs',
                      yaxis_title='Loss',
                      legend_title='Type')
  loss_fig.show()

  # Make predictions
  # Reshape X_test for prediction
  X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

  # Make predictions using the reshaped test data
  predictions = model.predict(X_test_reshaped)

  # Evaluate the model
  mse = mean_squared_error(y_test, predictions)
  mae = mean_absolute_error(y_test, predictions)
  r2 = r2_score(y_test, predictions)
  print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

  # Reshape y_test and predictions for inverse transform
  y_test_reshaped = y_test.values.reshape(-1, 1)
  predictions_reshaped = predictions.reshape(-1, 1)

  num_scaled_cols = 5

  # Create separate dummy arrays for inverse scaling
  dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
  dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

  # Fill in the 'Close' column values in the dummy arrays
  # Assuming 'Close' is the last of the scaled columns
  dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
  dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

  # Inverse transform the 'Close' prices using the dummy arrays
  y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
  predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

  # Extracting testing dates
  test_dates = test_data.index

  # Plotting with Plotly
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name=f'Actual {company_name}'))
  fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name=f'Predicted {company_name}'))
  fig.update_layout(title=f'Actual vs Predicted {company_name} Stock Prices',
                    xaxis_title='Date',
                    yaxis_title='Stock Price',
                    legend_title='Legend')
  fig.show()

  return model, history

In [29]:
msft_transformed_model = transformer_avg_model(msft_combined_data[0], msft_combined_data[1], 'MSFT', avg_weights)
ibm_transformed_model = transformer_avg_model(ibm_combined_data[0], ibm_combined_data[1], 'IBM', avg_weights)
amzn_transformed_model = transformer_avg_model(amzn_combined_data[0], amzn_combined_data[1], 'AMZN', avg_weights)
goog_transformed_model = transformer_avg_model(goog_combined_data[0], goog_combined_data[1], 'GOOG', avg_weights)
orcl_transformed_model = transformer_avg_model(orcl_combined_data[0], orcl_combined_data[1], 'ORCL', avg_weights)
nvda_transformed_model = transformer_avg_model(nvda_combined_data[0], nvda_combined_data[1], 'NVDA', avg_weights)
fdx_transformed_model = transformer_avg_model(fdx_combined_data[0], fdx_combined_data[1], 'FDX', avg_weights)
abnb_transformed_model = transformer_avg_model(abnb_combined_data[0], abnb_combined_data[1], 'ABNB', avg_weights)
tsla_transformed_model = transformer_avg_model(tsla_combined_data[0], tsla_combined_data[1], 'TSLA', avg_weights)
appl_transformed_model = transformer_avg_model(appl_combined_data[0], appl_combined_data[1], 'APPL', avg_weights)

Epoch 1/50
2/2 [==============================] - 3s 372ms/step - loss: 0.8568 - val_loss: 0.6072
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 0.5957 - val_loss: 0.6358
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 1.1755 - val_loss: 0.5476
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6016 - val_loss: 0.5402
Epoch 5/50
2/2 [==============================] - 0s 26ms/step - loss: 0.7447 - val_loss: 0.5386
Epoch 6/50
2/2 [==============================] - 0s 24ms/step - loss: 0.7377 - val_loss: 0.5269
Epoch 7/50
2/2 [==============================] - 0s 32ms/step - loss: 0.6217 - val_loss: 0.4303
Epoch 8/50
2/2 [==============================] - 0s 24ms/step - loss: 0.3882 - val_loss: 0.0628
Epoch 9/50
2/2 [==============================] - 0s 23ms/step - loss: 0.4484 - val_loss: 0.0986
Epoch 10/50
2/2 [==============================] - 0s 22ms/step - loss: 0.0616 - val_loss: 0.1005
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 292ms/step
MSE: 0.07539331791762421, MAE: 0.22186884214691846, R^2: 0.8092426946285965


Epoch 1/50
2/2 [==============================] - 2s 378ms/step - loss: 1.3514 - val_loss: 0.2198
Epoch 2/50
2/2 [==============================] - 0s 22ms/step - loss: 0.9406 - val_loss: 0.2721
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 0.6028 - val_loss: 0.1527
Epoch 4/50
2/2 [==============================] - 0s 31ms/step - loss: 0.5834 - val_loss: 0.0364
Epoch 5/50
2/2 [==============================] - 0s 25ms/step - loss: 0.5352 - val_loss: 0.0396
Epoch 6/50
2/2 [==============================] - 0s 28ms/step - loss: 0.2373 - val_loss: 0.0196
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 0.1594 - val_loss: 0.1005
Epoch 8/50
2/2 [==============================] - 0s 23ms/step - loss: 0.2229 - val_loss: 0.0323
Epoch 9/50
2/2 [==============================] - 0s 25ms/step - loss: 0.1156 - val_loss: 0.0768
Epoch 10/50
2/2 [==============================] - 0s 23ms/step - loss: 0.1785 - val_loss: 0.0284
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 295ms/step
MSE: 0.025716258017824775, MAE: 0.1366596093218051, R^2: -0.18290835566958874


Epoch 1/50
2/2 [==============================] - 3s 397ms/step - loss: 0.7377 - val_loss: 1.4002
Epoch 2/50
2/2 [==============================] - 0s 25ms/step - loss: 0.4377 - val_loss: 1.3359
Epoch 3/50
2/2 [==============================] - 0s 25ms/step - loss: 0.9882 - val_loss: 1.5557
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 0.4547 - val_loss: 1.4422
Epoch 5/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6141 - val_loss: 1.4513
Epoch 6/50
2/2 [==============================] - 0s 23ms/step - loss: 0.5985 - val_loss: 1.5234
Epoch 7/50
2/2 [==============================] - 0s 29ms/step - loss: 0.5328 - val_loss: 1.6856
Epoch 7: early stopping


...

1/1 [==============================] - 0s 306ms/step
MSE: 0.16534684274691047, MAE: 0.2883921163851568, R^2: 0.0380112597875506


Epoch 1/50
2/2 [==============================] - 2s 391ms/step - loss: 1.2003 - val_loss: 0.9660
Epoch 2/50
2/2 [==============================] - 0s 29ms/step - loss: 0.9228 - val_loss: 0.4264
Epoch 3/50
2/2 [==============================] - 0s 27ms/step - loss: 0.5316 - val_loss: 0.2177
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 0.3117 - val_loss: 0.7496
Epoch 5/50
2/2 [==============================] - 0s 28ms/step - loss: 0.8531 - val_loss: 0.0163
Epoch 6/50
2/2 [==============================] - 0s 30ms/step - loss: 0.2602 - val_loss: 0.7557
Epoch 7/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6100 - val_loss: 0.8774
Epoch 8/50
2/2 [==============================] - 0s 25ms/step - loss: 0.4324 - val_loss: 0.7965
Epoch 9/50
2/2 [==============================] - 0s 24ms/step - loss: 0.6251 - val_loss: 0.8815
Epoch 10/50
2/2 [==============================] - 0s 31ms/step - loss: 0.4666 - val_loss: 0.8560
Epoch 10: early stopping


1/1 [==============================] - 0s 298ms/step
MSE: 0.043430693561766674, MAE: 0.1692612688461488, R^2: 0.43489487023557494


Epoch 1/50
2/2 [==============================] - 2s 361ms/step - loss: 1.4057 - val_loss: 0.0513
Epoch 2/50
2/2 [==============================] - 0s 29ms/step - loss: 1.3876 - val_loss: 0.0514
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 1.2317 - val_loss: 0.2327
Epoch 4/50
2/2 [==============================] - 0s 26ms/step - loss: 1.1612 - val_loss: 0.1032
Epoch 5/50
2/2 [==============================] - 0s 29ms/step - loss: 0.4966 - val_loss: 0.0238
Epoch 6/50
2/2 [==============================] - 0s 27ms/step - loss: 0.8556 - val_loss: 0.0860
Epoch 7/50
2/2 [==============================] - 0s 25ms/step - loss: 0.7475 - val_loss: 0.2003
Epoch 8/50
2/2 [==============================] - 0s 24ms/step - loss: 0.7026 - val_loss: 0.0303
Epoch 9/50
2/2 [==============================] - 0s 28ms/step - loss: 0.4182 - val_loss: 0.0416
Epoch 10/50
2/2 [==============================] - 0s 30ms/step - loss: 0.2395 - val_loss: 0.1594
Epoch 10: early stopping


1/1 [==============================] - 0s 304ms/step
MSE: 0.19561935160990795, MAE: 0.3303314443950066, R^2: -0.1825095469588074


In [31]:
import matplotlib.pyplot as plt

num_iterations = 10  # Define the number of iterations for the process
all_train_losses = []
all_val_losses = []

for iteration in range(num_iterations):
    # Average the weights
    avg_weights = average_model_weights(existing_models)

    iteration_train_losses = []
    iteration_val_losses = []

    # Retrain models for each company
    for company_data, company_name in zip([msft_combined_data, ibm_combined_data, amzn_combined_data, goog_combined_data, orcl_combined_data, nvda_combined_data, fdx_combined_data, abnb_combined_data, tsla_combined_data, appl_combined_data], ['MSFT', 'IBM', 'AMZN', 'GOOG', 'ORCL', 'NVDA', 'FDX', 'ABNB', 'TSLA', 'APPL']):
        model, history = transformer_avg_model(company_data[0], company_data[1], company_name, avg_weights)

        # Record the losses for this iteration
        iteration_train_losses.extend(history.history['loss'])
        iteration_val_losses.extend(history.history['val_loss'])

        # Update the existing_models list with the retrained model
        if company_name == 'MSFT':
            msft_model = model
        elif company_name == 'IBM':
            ibm_model = model
        elif company_name == 'AMZN':
            amzn_model = model
        elif company_name == 'GOOG':
            goog_model = model
        elif company_name == 'ORCL':
            orcl_model = model
        elif company_name == 'NVDA':
            nvda_model = model
        elif company_name == 'FDX':
            fdx_model = model
        elif company_name == 'ABNB':
            abnb_model = model
        elif company_name == 'TSLA':
            tsla_model = model
        elif company_name == 'APPL':
            appl_model = model

    existing_models = [msft_model, ibm_model, amzn_model, goog_model, orcl_model, nvda_model, fdx_model, abnb_model, tsla_model, appl_model]  # Update the list with retrained models

    # Calculate average losses for this iteration
    avg_train_loss = np.mean(iteration_train_losses)
    avg_val_loss = np.mean(iteration_val_losses)

    all_train_losses.append(avg_train_loss)
    all_val_losses.append(avg_val_loss)

    existing_models = [msft_model, ibm_model, amzn_model, goog_model, orcl_model, nvda_model, fdx_model, abnb_model, tsla_model, appl_model]  # Update the list with retrained models


# Creating a figure
fig = go.Figure()

# Adding the training loss plot
fig.add_trace(go.Scatter(
    x=list(range(num_iterations)),
    y=all_train_losses,
    mode='lines+markers',
    name='Average Training Loss'
))

# Adding the validation loss plot
fig.add_trace(go.Scatter(
    x=list(range(num_iterations)),
    y=all_val_losses,
    mode='lines+markers',
    name='Average Validation Loss'
))

# Updating the layout of the figure
fig.update_layout(
    title='Average Training and Validation Loss Across Iterations',
    xaxis_title='Iteration',
    yaxis_title='Loss',
    legend_title='Legend',
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

# Show the figure
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [32]:
# Creating a figure
fig = go.Figure()

# Adding the training loss plot
fig.add_trace(go.Scatter(
    x=list(range(num_iterations)),
    y=all_train_losses,
    mode='lines+markers',
    name='Average Training Loss'
))

# Adding the validation loss plot
fig.add_trace(go.Scatter(
    x=list(range(num_iterations)),
    y=all_val_losses,
    mode='lines+markers',
    name='Average Validation Loss'
))

# Updating the layout of the figure
fig.update_layout(
    title='Average Training and Validation Loss Across Iterations',
    xaxis_title='Iteration',
    yaxis_title='Loss',
    legend_title='Legend',
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

# Show the figure
fig.show()

# Separating via R2 Scores

In [33]:
import numpy as np

# Assuming you have these models already trained
existing_models = [msft_model, ibm_model, amzn_model, goog_model, orcl_model, nvda_model, fdx_model, abnb_model, tsla_model, appl_model]

def average_model_weights(models):
    # Filter out any None values from the models list
    valid_models = [model for model in models if model is not None]

    if not valid_models:
        raise ValueError("No valid models provided for averaging weights.")

    num_layers = len(valid_models[0].layers)
    average_weights = []

    for layer in range(num_layers):
        layer_weights_all_models = []

        # Collect weights for this layer from all models
        for model in valid_models:
            layer_weights = model.layers[layer].get_weights()
            if layer_weights:  # Ensure there are weights to process
                layer_weights_all_models.append(layer_weights)

        if not layer_weights_all_models:
            continue  # Skip layers with no weights

        # Calculate the average weights for this layer
        num_weights = len(layer_weights_all_models[0])  # Number of weight arrays in the layer (usually 2: weights and biases)
        avg_layer_weights = []
        for i in range(num_weights):
            # Calculate the mean for each weight array across all models
            weights_i = np.mean([model_weights[i] for model_weights in layer_weights_all_models], axis=0)
            avg_layer_weights.append(weights_i)

        average_weights.append(avg_layer_weights)

    # Flatten the list of averaged weights for setting in a model
    average_weights_flat = [weight for layer_weights in average_weights for weight in layer_weights]
    return average_weights_flat

avg_weights = average_model_weights(existing_models)

def transformer_avg_model(combined_data, scaler_prices, company_name, avg_weights):
  selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
  data = combined_data[selected_features]

  train_size = int(len(data) * 0.7)
  val_size = int(len(data) * 0.15)
  train_data = data.iloc[:train_size]
  val_data = data.iloc[train_size:train_size + val_size]
  test_data = data.iloc[train_size + val_size:]

  # Prepare features and target for model training
  X_train = train_data.drop('Close', axis=1)
  y_train = train_data['Close']
  X_val = val_data.drop('Close', axis=1)
  y_val = val_data['Close']
  X_test = test_data.drop('Close', axis=1)
  y_test = test_data['Close']

  # Define and compile the neural network model
  def create_model(input_shape):
      inp = Input(shape=input_shape)

      # LSTM Layer
      x = LSTM(128, return_sequences=True)(inp)
      x = Dropout(0.2)(x)

      # Transformer Layer using MultiHeadAttention
      attention = MultiHeadAttention(num_heads=2, key_dim=128)(x, x)
      x = GlobalAveragePooling1D()(attention)

      # Dense Layers for final predictions
      x = Dense(64, activation='LeakyReLU')(x)
      x = Dropout(0.2)(x)
      x = Dense(32, activation='LeakyReLU')(x)
      x = Dense(1)(x)

      model = Model(inputs=inp, outputs=x)
      model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
      return model

  # Reshape the data for LSTM layer
  X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
  X_val_reshaped = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))
  X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

  # Train the model
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

  # Train the model with early stopping
  model = create_model((1, X_train.shape[1]))
  model.set_weights(avg_weights)
  history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=8, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])

  train_loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(train_loss) + 1)

  # Convert the epochs range object to a list
  epochs_list = list(epochs)

  # Plotting the training and validation loss
  loss_fig = go.Figure()
  loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name=f'{company_name} Training Loss'))
  loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name=f'{company_name} Validation Loss'))
  loss_fig.update_layout(title=f'{company_name} Training and Validation Loss per Epoch',
                      xaxis_title='Epochs',
                      yaxis_title='Loss',
                      legend_title='Type')
  loss_fig.show()

  # Make predictions
  # Reshape X_test for prediction
  X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

  # Make predictions using the reshaped test data
  predictions = model.predict(X_test_reshaped)

  # Evaluate the model
  mse = mean_squared_error(y_test, predictions)
  mae = mean_absolute_error(y_test, predictions)
  r2 = r2_score(y_test, predictions)
  print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

  # Reshape y_test and predictions for inverse transform
  y_test_reshaped = y_test.values.reshape(-1, 1)
  predictions_reshaped = predictions.reshape(-1, 1)

  num_scaled_cols = 5

  # Create separate dummy arrays for inverse scaling
  dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
  dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

  # Fill in the 'Close' column values in the dummy arrays
  # Assuming 'Close' is the last of the scaled columns
  dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
  dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

  # Inverse transform the 'Close' prices using the dummy arrays
  y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
  predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

  # Extracting testing dates
  test_dates = test_data.index

  # Plotting with Plotly
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name=f'Actual {company_name}'))
  fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name=f'Predicted {company_name}'))
  fig.update_layout(title=f'Actual vs Predicted {company_name} Stock Prices',
                    xaxis_title='Date',
                    yaxis_title='Stock Price',
                    legend_title='Legend')
  fig.show()

  return model, mse, mae, r2

In [38]:
import numpy as np
import plotly.graph_objects as go

# Assuming company_data and transformer_avg_model are properly defined elsewhere

# Configuration
r_squared_threshold = 0.75
group_a_companies = []  # Companies with R^2 >= threshold
group_b_companies = []  # Companies with R^2 < threshold

# Placeholder for storing losses
group_a_losses = {'train': [], 'val': []}
group_b_losses = {'train': [], 'val': []}

company_data = {
    'MSFT': msft_combined_data,
    'IBM': ibm_combined_data,
    'AMZN': amzn_combined_data,
    'GOOG': goog_combined_data,
    'ORCL': orcl_combined_data,
    'NVDA': nvda_combined_data,
    'FDX': fdx_combined_data,
    'ABNB': abnb_combined_data,
    'TSLA': tsla_combined_data,
    'APPL': appl_combined_data,
}


results = {}  # Store results for further processing or analysis

for company_name, combined_data in company_data.items():
    model, mse, mae, r2 = transformer_avg_model(combined_data[0], combined_data[1], company_name, avg_weights)
    # Decide what to do based on R^2 value
    if r2 >= r_squared_threshold:
        group_a_companies.append((company_name, model))
    else:
        group_b_companies.append((company_name, model))

    # Store results
    results[company_name] = {
        "model": model,
        "history": history,
        "R^2": r2
    }

# Function to process and plot losses for groups
def process_and_plot_group_losses(group_name, companies):
    # Assuming comp[2] is a History object with a .history attribute
    avg_train_losses = np.mean([comp[2].history['loss'] for comp in companies])
    avg_val_losses = np.mean([comp[2].history['val_loss'] for comp in companies])

    fig.add_trace(go.Scatter(
        x=list(range(1, len(companies) + 1)),
        y=[avg_train_losses] * len(companies),
        mode='lines+markers',
        name=f'Group {group_name} Average Training Loss'
    ))
    fig.add_trace(go.Scatter(
        x=list(range(1, len(companies) + 1)),
        y=[avg_val_losses] * len(companies),
        mode='lines+markers',
        name=f'Group {group_name} Average Validation Loss'
    ))

# Assuming average_model_weights function and transformer_avg_model are defined and work correctly

fig = go.Figure()

# Process and plot losses for both groups
process_and_plot_group_losses('A', group_a_companies)
process_and_plot_group_losses('B', group_b_companies)

# Updating the layout of the figure
fig.update_layout(
    title='Average Training and Validation Loss for Groups A and B',
    xaxis_title='Company Index',
    yaxis_title='Loss',
    legend_title='Legend',
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

fig.show()


Epoch 1/50
2/2 [==============================] - 4s 386ms/step - loss: 2.8808 - val_loss: 0.0459
Epoch 2/50
2/2 [==============================] - 0s 24ms/step - loss: 0.2979 - val_loss: 0.3531
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 0.7534 - val_loss: 0.4192
Epoch 4/50
2/2 [==============================] - 0s 22ms/step - loss: 0.7894 - val_loss: 0.4353
Epoch 5/50
2/2 [==============================] - 0s 23ms/step - loss: 0.7787 - val_loss: 0.4378
Epoch 6/50
2/2 [==============================] - 0s 33ms/step - loss: 0.7057 - val_loss: 0.4055
Epoch 6: early stopping


1/1 [==============================] - 0s 309ms/step
MSE: 0.055883952223346474, MAE: 0.2016389776234023, R^2: 0.8586045496594621


Epoch 1/50
2/2 [==============================] - 2s 531ms/step - loss: 7.2581 - val_loss: 0.0217
Epoch 2/50
2/2 [==============================] - 0s 32ms/step - loss: 0.4633 - val_loss: 0.0764
Epoch 3/50
2/2 [==============================] - 0s 39ms/step - loss: 1.2186 - val_loss: 0.1002
Epoch 4/50
2/2 [==============================] - 0s 36ms/step - loss: 1.2836 - val_loss: 0.1072
Epoch 5/50
2/2 [==============================] - 0s 55ms/step - loss: 1.3139 - val_loss: 0.1154
Epoch 6/50
2/2 [==============================] - 0s 40ms/step - loss: 1.2760 - val_loss: 0.1228
Epoch 6: early stopping


1/1 [==============================] - 0s 460ms/step
MSE: 0.014058124561709824, MAE: 0.1051522300760287, R^2: 0.3533478705352856


Epoch 1/50
2/2 [==============================] - 2s 371ms/step - loss: 4.5375 - val_loss: 0.0040
Epoch 2/50
2/2 [==============================] - 0s 45ms/step - loss: 0.1899 - val_loss: 0.9290
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 0.6931 - val_loss: 1.1183
Epoch 4/50
2/2 [==============================] - 0s 24ms/step - loss: 0.7208 - val_loss: 1.1874
Epoch 5/50
2/2 [==============================] - 0s 25ms/step - loss: 0.7054 - val_loss: 1.2661
Epoch 6/50
2/2 [==============================] - 0s 31ms/step - loss: 0.6762 - val_loss: 1.3361
Epoch 6: early stopping


...

Epoch 1/50
2/2 [==============================] - 2s 381ms/step - loss: 7.4761 - val_loss: 0.0192
Epoch 2/50
2/2 [==============================] - 0s 28ms/step - loss: 0.2829 - val_loss: 0.5232
Epoch 3/50
2/2 [==============================] - 0s 26ms/step - loss: 1.1357 - val_loss: 0.6185
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 1.2113 - val_loss: 0.6482
Epoch 5/50
2/2 [==============================] - 0s 28ms/step - loss: 1.2144 - val_loss: 0.6854
Epoch 6/50
2/2 [==============================] - 0s 53ms/step - loss: 1.1430 - val_loss: 0.7275
Epoch 6: early stopping


1/1 [==============================] - 0s 324ms/step
MSE: 0.020932658577422346, MAE: 0.125702811889959, R^2: 0.7276315027093601


Epoch 1/50
2/2 [==============================] - 2s 384ms/step - loss: 9.1113 - val_loss: 0.0438
Epoch 2/50
2/2 [==============================] - 0s 25ms/step - loss: 0.3152 - val_loss: 0.0450
Epoch 3/50
2/2 [==============================] - 0s 25ms/step - loss: 1.2546 - val_loss: 0.0471
Epoch 4/50
2/2 [==============================] - 0s 25ms/step - loss: 1.3312 - val_loss: 0.0477
Epoch 5/50
2/2 [==============================] - 0s 25ms/step - loss: 1.3511 - val_loss: 0.0476
Epoch 6/50
2/2 [==============================] - 0s 33ms/step - loss: 1.3408 - val_loss: 0.0471
Epoch 6: early stopping


1/1 [==============================] - 0s 313ms/step
MSE: 0.06204300715483544, MAE: 0.24544241105136363, R^2: 0.6249530137032182


IndexError: tuple index out of range